# Unsupervised Learning Techniques

**Clustering**: grouping similar instances into clusters.

**Anomaly detection**: objective to learn what "normal" data looks like, and use it to detect abnormal instances.

**Novelty detection**

**Density estimation**: estimating the probability density function (PDF) of the random process that generated the dataset. Useful for data analysis, visualization and detecting anomalies.

# Clustering
Works well with multiple features.

## K-Means

```python
from sklearn.cluster import KMeans

k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

X_new = ...
kmeans.predict(X_new)

# measure distance from each instance to every centroid
kmeans.transform(X_new)
```

Properties:
- An instance's *label* is the index of the cluster that the instance gets assigned to.

You can plot the decision boundaries to get a *Voronoi tessellation*.

**Hard clustering**: assigning each instance to a single cluster.

**Soft clustering**: giving each instance a score per cluster.

**Normal Algorithm**:
- Place centroid randomly, label instances, update centroid, label, etc. until centroids stop moving.
- Can get stuck into a local optimum if you are not lucky with the random initialization step.
- sklearn uses the `n_init` parameter to run the model multiple times and keep the model with the best performance (lowest inertia). The `score` is negative, as it respects the *"great is better"* rule.

**K-Means++**:
- Take one centroid $c^{(1)}$, chosen uniformly at random from the dataset.
- Take a new centroid $c^{(i)}$, choosing an instance $x^{(i)}$ with probability $D(x^{(i)})^2 \Sigma^m_{j=1}D(x^{(j)})^2$ where $D(x^{(i)}$ is the distance between the instance $x^{(i)}$ and the closest centroid that was already chose. This distribution ensures that instances further away from already chosen centroids are more likely to be selected as centroids.
- Repeat until all $k$ centroids have been chosen.

**MiniBatchKMeans**:
- Speed up KMeans by a factor 3/4 and make it posible to cluster huge data.
- Intertia is generally slightly worse.
- `partial_fit` can be used, but using `memmap` class is easiest.

```python

from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=k)
minibatch_kmeans.fit(X)
```

**Selecting number of clusters**:
- Use elbow rule, as picking the lowest inertia won't make sense, as it will keep decreasing.
    - Rough estimation.
- Silhouette score.
    - More precise (but more computationally expensive)
    - Mean of the *silhouette coefficient*.
    - *silhouette coefficient*: $(b-a)/max(a,b)$ where $a$ is mean distance to other instances in the same cluster, and $b$ is the mean nearest-cluster distance.
    - Score between -1 and +1. 
        - Close to +1 means the instance is well inside its own cluster and far from other clusters.
        - 0 means that is close to a cluster boundary.
        - -1 means that the instance may have been assigned to the wrong cluster.

```python
from sklearn.metrics import silhoutte_score

silhouette_score(X, kmeans.labels_)

```

- Silhouette diagram: plotting every instance's silhouette coefficient, sorted by the cluster they are assigned to and by value of the coefficient.
    - Vertical dashed line is the silhouette score for each number of clusters.
    - if most instances in a cluster have a lower coefficient than the score, it is bad.
    - Better if bigger than score and of similar size.
    
**Limits of K-means**:
- Run algorithm several times to avoid sub-optimal solutions.
- Does not behave well when clusters have varying sizes, densities or non-spherical shape.

Important to scale the input features before running K-means!



In [6]:
# example: clustering for preprocessing
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

X_digits, y_digits = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

log_reg = LogisticRegression(random_state=42, max_iter=3000)
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.9777777777777777

In [7]:
pipeline = Pipeline([
    ('kmeans', KMeans(n_clusters=50)),
    ('log_reg', LogisticRegression(random_state=42, max_iter=3000))
])
pipeline.fit(X_train, y_train)

pipeline.score(X_test, y_test)

C:\Users\ddlat\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(
C:\Users\ddlat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9822222222222222

In [8]:
param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits


C:\Users\ddlat\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.4s


C:\Users\ddlat\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.6s


...

[CV] END ..............................kmeans__n_clusters=99; total time=   3.2s


C:\Users\ddlat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ddlat\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


[CV] END ..............................kmeans__n_clusters=99; total time=   3.3s


C:\Users\ddlat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg',
                                        LogisticRegression(max_iter=3000,
                                                           random_state=42))]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [9]:
grid_clf.best_params_

{'kmeans__n_clusters': 92}

In [10]:
grid_clf.score(X_test, y_test)

0.9888888888888889

In [12]:
# example: clustering for semi-supervised learning
n_labeled= 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])
log_reg.score(X_test, y_test)

C:\Users\ddlat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8644444444444445

In [13]:
# transform data and manually label it
# you can propagate the labels to values near the centroid

## DBSCAN
The algorthm defines clusters as continuous regions of high density.

**Algorithm**:
- For each instance, count how many instances are located within a small distance $\epsilon$ from it. This region is the instance's $\epsilon$-neighbourhood.
- If an instance has at least min_samples instances in its $\epsilon$-neighbourhood (including itself), then it is considered a core instance.
- All instances in the neighbourhood of a core instance belongs to the same cluster. Therefore, a long sequence of neigbouring core instances form a single cluster.
- Any instance that is not a core instance and does not have one in its neighbourhood is considered an anomaly.

```python

from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

```

**Labels**:
- Instances with a cluster index of $-1$ are considered anomalies.

**Notes**:
- The class has no predict function as the rationale is that several classification algorithms could make sense, using the components and labels.

**Pros and Cons**:
- Capable of identifying any number of clusters.
- Capable of any shape.
- Robust to outliers.
- Density varies across clusters and it can be impossible to capture all clusters properly.
- Computational complexity of $O(m log m)$ which is close to linear and can go up to $O(M^2)$.

## Other algorithms
**Agglomerative clustering**: a hierarchy of clusters is built from the bottom up.
- Scales well when using a connectivity matrix
- Capture various shapes
- Produces a cluster tree
- Can use any pairwise distance

**Birch**: builds a tree structure during training containing just enough information to assign each new instance to a cluster, without having to store all instances in the tree.
- n_features < 20

**Mean-shift**: starts by placing a circle centered on each instance, then for each circle it computtes the mean of all isntances located within it, and it shifts the circle so that it is centered on the mean. It iterates thhis mean0shift step until all circles stop moving.
- Any shape
- Relies on local density estimation
- Complexity is $O(m^2)$.

**Affinity propagation**: uses a voting system, where instances vote for similar instances to be their representatives.
- detect any number of clusters of different sizes.
- complexity is $O(m^2)$.

**Spectral clustering**: takes a similarity matrix and creates a low-dimensional embedding from it, then uses another clustering algorithm. 
- Captures complex cluster structures.
- Can be used to cut graphs.
- Does not scale well.
- Does not behave well when clusters have different sizes.

## Gaussian Mixture Model (GMM)
A probabilistic model that assumes that the instances were generated from a a mixture of several Gaussian distributions whose parameters are unknown.

All instances generated by a single Gaussian distribution form a cluster that typically looks like an ellipsoid.

It uses Expectectation-Maximization EM algorithm, soft cluster assignment but is prone to local minima. Therefore set `n_init` to `10`.

```python

from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

```

It is a generative model, thus you can sample new instances from it!

You can reduce difficulty by limiting number of parameters, e.g. shapes:
- `covariance_type`: `spherical` all clusters must be spherical, `diag` ellipsoidal shape parallel to coordinate axes, `tied` same ellipsoidal shape, size and orientation.

**Notes**:
- Doesn't scale well to large number of features
- Tries to fit all the data, including outloiers, so if you have too many, this will bias the model's view of "normality". Can clean up and try to use `Ell1pt1Envelope`


**Anomaly detection**: any instance located in a low-density region can be considered an anomaly. Requires a threshold.

```python

densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]

```

**Novelty detection**: assumed to be trained on a clean dataset, uncontaminated by outlies, where anomaly detection does not make this assumption.

**Selecting the number of clusters**
- GMM does not provide inertia or silhouette score, because it's not reliable if clusters are not spherical or have different sizes.
- Minimize a *theoretical information criterion* such as *bayesian information criterion (BIC)* or *Akaike information criterion (AIC).

$$
BIC = log(m)p-2log(\hat L)
$$
$$
AIC = 2p - 2log(\hat L)
$$

Where:
- $m$ is the number of instances
- $p$ is the number of parameters learned by the model
- $\hat L$ is the maximized value of the likelihood function of the model

```python

gm.bic(X)

gm.aic(X)

```

Also able to use Bayesian gaussian mixture models which automatically find the optimal number of clusters < n_components

```python

from sklearn.mixture import BayesianGaussianMixture

bgm = BayesianGaussianMixture(n_components=10, n_init=10, random_state=42)
bgm.fit(X)

```

## Other Anomaly detection and Novely detection algorithms
- Fast-MCD (minimum covariance terminant). Useful for outlier detection to cleanup a dataset. Assument inliers are gaussian and that outliers were not generated from this guassian distribution.
- Isolation forest: outlier detection in high-dimensional datasets.
- Local outlier factor (LOF): good for outlier detection. Compares density. Anomaly is often more isolated than its k-nearest neigbours.
- One-class SVM: suited for novelty detection. 